In [1]:
import pandas as pd
from pathlib import Path
import socket
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import pyreadr
import pickle

In [2]:
#paths
fig_path = Path("../figures")
df_path = Path("../dataframes")

custom_colorscale = [
    [0.0, "white"],      # 0 maps to white
    [0.00001, "#440154"],  # start of Viridis
    [0.25, "#3b528b"],
    [0.5, "#21918c"],
    [0.75, "#5ec962"],
    [1.0, "#fde725"]
]
custom_colorscale = "Viridis"

In [3]:
def find_free_port(start_port=8050, max_port=8100):
    port = start_port
    while port <= max_port:
        with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
            try:
                s.bind(("127.0.0.1", port))
                return port
            except OSError:
                port += 1
    raise RuntimeError("No free ports available")

def update_city_plot(city_name, duration_label):
    df = dataframes[city_name]
    duration_value = int(duration_label.split()[0])
    df_sub = df[df['duration'] == duration_value]

    fig = px.line(
        df_sub,
        x='year',
        y='nduration',
        color='ensemble',
        markers=True,
        title=f"{duration_label} Heatwave Events at {city_name.replace('_', ' ')}"
    )

    #mean 
    for trace in fig.data:
        if trace.name == "mean":
            trace.line.color = "black"
            trace.line.dash = "dash"

    fig.update_layout(
        xaxis_title='Year',
        yaxis_title='nDuration',
        height=400
    )

    return fig


In [4]:
#read in duration df
duration_df = pd.read_csv(df_path / "duration_df.csv")

# Load the dataframes dictionary from the file
with open("../dataframes/duration_dataframes_dict.pkl", "rb") as file:
    dataframes = pickle.load(file)
#print(dataframes["London"])
#read in city df and get wanted cities
result = pyreadr.read_r('/data/users/laura.owen/extremes/heatwaves/HadUKGrid/dur-clim/coords/UK_top30_cities.Rda')
city_df = result['city_df']
city_names_in_dataframes = set(dataframes.keys())
city_df = city_df[city_df['city'].isin(city_names_in_dataframes)].copy()

In [5]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Heatwave Duration (days): 1980 vs 2080"),
    
    dcc.Dropdown(
        id='duration-dropdown',
        options=[{'label': f'{i} day', 'value': i} for i in range(1, 10)],
        value=1,
        clearable=False
    ),

    html.Div([
        # Left column: two heatmaps side-by-side
        html.Div([
            dcc.Graph(id='heatmap-1980', style={'height': '80vh', 'width': '48%'}),
            dcc.Graph(id='heatmap-2080', style={'height': '80vh', 'width': '48%'}),
        ], style={'display': 'flex', 'flexDirection': 'row', 'gap': '10px', 'flex': '1'}),

        # Right column: timeseries plot fills height
        html.Div([
            dcc.Dropdown(
                id='city-dropdown',
                options=[{'label': name.replace("_", " "), 'value': name} for name in dataframes.keys()],
                value=None,
                placeholder="Select a city",
                clearable=True
            ),
            dcc.Graph(id='timeseries-plot', style={'height': '90vh', 'width': '100%'})
        ], style={'flex': '1', 'marginLeft': '20px'})
    ], style={'display': 'flex', 'flexDirection': 'row', 'height': '80vh'}),
])

@app.callback(
    [Output('heatmap-1980', 'figure'),
     Output('heatmap-2080', 'figure')],
    Input('duration-dropdown', 'value')
)

def update_heatmaps(selected_duration):
    col_1980 = f'duration{selected_duration}_1980'
    col_2080 = f'duration{selected_duration}_2080'

    combined_min = min(duration_df[col_1980].min(), duration_df[col_2080].min())
    combined_max = max(duration_df[col_1980].max(), duration_df[col_2080].max())

    fig_1980 = px.density_heatmap(
        duration_df, x='xco', y='yco', z=col_1980,
        nbinsx=131, nbinsy=211,
        color_continuous_scale=custom_colorscale,
        range_color=[combined_min, combined_max],
        title=f'{selected_duration} day duration - 1980'
    )
    fig_1980.update_layout(
        xaxis_title='lon', yaxis_title='lat',
        coloraxis_colorbar=dict(title="nDuration"),
    )
    fig_1980.add_trace(go.Scatter(
        x=city_df["lon_index"], y=city_df["lat_index"],
        mode='markers',
        marker=dict(color='yellow', size=8),
        text=city_df["city"],
        customdata=city_df["city"],
        name='Cities',
        hoverinfo='text',
    ))

    fig_2080 = px.density_heatmap(
        duration_df, x='xco', y='yco', z=col_2080,
        nbinsx=131, nbinsy=211,
        color_continuous_scale=custom_colorscale,
        range_color=[combined_min, combined_max],
        title=f'{selected_duration} day duration - 2080'
    )
    fig_2080.update_layout(
        xaxis_title='lon', yaxis_title='lat',
        coloraxis_colorbar=dict(title="nDuration"),
    )
    fig_2080.add_trace(go.Scatter(
        x=city_df["lon_index"], y=city_df["lat_index"],
        mode='markers',
        marker=dict(color='yellow', size=8),
        text=city_df["city"],
        customdata=city_df["city"],
        name='Cities',
        hoverinfo='text',
    ))

    return fig_1980, fig_2080

@app.callback(
    Output('city-dropdown', 'value'),
    Input('heatmap-1980', 'clickData'),
    prevent_initial_call=True
)

def update_city_from_click(clickData):
    if clickData and 'points' in clickData:
        return clickData['points'][0].get('customdata')
    return None

@app.callback(
    Output('timeseries-plot', 'figure'),
    [Input('heatmap-1980', 'clickData'),
     Input('city-dropdown', 'value'),
     Input('duration-dropdown', 'value')]
)

def update_timeseries(clickData, dropdown_city, selected_duration):
    # Determine city from clickData or dropdown
    city_name = None
    if clickData and 'points' in clickData and clickData['points'][0].get('customdata'):
        city_name = clickData['points'][0]['customdata']
    elif dropdown_city:
        city_name = dropdown_city

    if not city_name or not selected_duration:
        return go.Figure()

    duration_label = f"{selected_duration} day"
    return update_city_plot(city_name, duration_label)

if __name__ == '__main__':
    free_port = find_free_port()
    url = f"http://127.0.0.1:{free_port}"
    print(f"Starting Dash app at: {url}")
    app.run(debug=True, port=free_port)




Starting Dash app at: http://127.0.0.1:8059
